# Data Cleaning & Preprocessing
## Global Weather Repository Analysis

**Goal**: Prepare the dataset for analysis by handling data types, creating useful features, and organizing the data properly.

---

In [1]:
# load what we need
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# make plots look better
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

print('Libraries loaded successfully!')

Libraries loaded successfully!


In [2]:
# load the data
df = pd.read_csv('../data/GlobalWeatherRepository.csv')

print(f'Loaded {len(df):,} weather records from {df["country"].nunique()} countries')
print(f'Data covers {df["location_name"].nunique()} unique locations worldwide')

df.head(3)

Loaded 103,868 weather records from 211 countries
Data covers 254 unique locations worldwide


,country,location_name,latitude,longitude,timezone,last_updated_epoch,last_updated,temperature_celsius,temperature_fahrenheit,condition_text,...,air_quality_PM2.5,air_quality_PM10,air_quality_us-epa-index,air_quality_gb-defra-index,sunrise,sunset,moonrise,moonset,moon_phase,moon_illumination
0,Afghanistan,Kabul,34.52,69.18,Asia/Kabul,1715849100,2024-05-16 13:15,26.6,79.8,Partly Cloudy,...,8.4,26.6,1,1,04:50 AM,06:50 PM,12:12 PM,01:11 AM,Waxing Gibbous,55
1,Albania,Tirana,41.33,19.82,Europe/Tirane,1715849100,2024-05-16 10:45,19.0,66.2,Partly cloudy,...,1.1,2.0,1,1,05:21 AM,07:54 PM,12:58 PM,02:14 AM,Waxing Gibbous,55
2,Algeria,Algiers,36.76,3.05,Africa/Algiers,1715849100,2024-05-16 09:45,23.0,73.4,Sunny,...,10.4,18.4,1,1,05:40 AM,07:50 PM,01:15 PM,02:14 AM,Waxing Gibbous,55


## Quick Data Quality Check

Even though the initial inspection showed no missing values, let's verify and understand our data better.

In [3]:
# check data types and memory
print('Dataset Info:')
print(f'Shape: {df.shape}')
print(f'Memory: {df.memory_usage(deep=True).sum() / 1024**2:.1f} MB')
print(f'\nMissing values: {df.isnull().sum().sum()}')
print(f'Duplicate rows: {df.duplicated().sum()}')

Dataset Info:
Shape: (103868, 41)
Memory: 87.6 MB

Missing values: 0
Duplicate rows: 0


## Fix Data Types & Parse Dates

The timestamps are stored as strings - we need to convert them to datetime objects for time series analysis.

In [4]:
# convert timestamp columns to datetime
df['last_updated'] = pd.to_datetime(df['last_updated'])

# also convert sunrise/sunset times
# note: these might not have dates, just times - we'll handle that
print('Converted last_updated to datetime format')
print(f'Date range: {df["last_updated"].min()} to {df["last_updated"].max()}')

# extract useful time features
df['year'] = df['last_updated'].dt.year
df['month'] = df['last_updated'].dt.month
df['day'] = df['last_updated'].dt.day
df['hour'] = df['last_updated'].dt.hour
df['day_of_week'] = df['last_updated'].dt.dayofweek  # 0=Monday, 6=Sunday
df['day_name'] = df['last_updated'].dt.day_name()

print('\nCreated time-based features: year, month, day, hour, day_of_week')

Converted last_updated to datetime format
Date range: 2024-05-16 01:45:00 to 2025-11-01 21:15:00

Created time-based features: year, month, day, hour, day_of_week


In [5]:
# quick check of temporal distribution
print('Data distribution over time:')
print(df['year'].value_counts().sort_index())
print(f'\nMonths covered: {sorted(df["month"].unique())}')

Data distribution over time:
year
2024    44469
2025    59399
Name: count, dtype: int64

Months covered: [np.int32(1), np.int32(2), np.int32(3), np.int32(4), np.int32(5), np.int32(6), np.int32(7), np.int32(8), np.int32(9), np.int32(10), np.int32(11), np.int32(12)]


## Create Geographic Features

Let's add hemisphere and region information based on coordinates.

In [6]:
# determine hemisphere
df['hemisphere'] = df['latitude'].apply(lambda x: 'Northern' if x >= 0 else 'Southern')

# categorize by latitude zones (rough climate zones)
def get_climate_zone(lat):
    lat = abs(lat)
    if lat >= 66.5:
        return 'Polar'
    elif lat >= 35:
        return 'Temperate'
    elif lat >= 23.5:
        return 'Subtropical'
    else:
        return 'Tropical'

df['climate_zone'] = df['latitude'].apply(get_climate_zone)

print('Geographic features created:')
print(f'\nHemisphere distribution:')
print(df['hemisphere'].value_counts())
print(f'\nClimate zones:')
print(df['climate_zone'].value_counts())

Geographic features created:

Hemisphere distribution:
hemisphere
Northern    81035
Southern    22833
Name: count, dtype: int64

Climate zones:
climate_zone
Tropical       51588
Temperate      36895
Subtropical    15385
Name: count, dtype: int64


## Handle Outliers

Let's check for extreme values that might be errors or need special attention.

In [7]:
# function to detect outliers using IQR method
def find_outliers(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 3 * IQR  # using 3*IQR for weather (more lenient than 1.5)
    upper = Q3 + 3 * IQR
    outliers = data[(data[column] < lower) | (data[column] > upper)]
    return len(outliers), lower, upper

# check key weather variables
columns_to_check = ['temperature_celsius', 'humidity', 'wind_kph', 'pressure_mb']

print('Outlier Detection (using 3*IQR method):')
print('='*60)
for col in columns_to_check:
    n_outliers, lower, upper = find_outliers(df, col)
    pct = (n_outliers / len(df)) * 100
    print(f'{col}:')
    print(f'  Outliers: {n_outliers} ({pct:.2f}%)')
    print(f'  Range: [{lower:.1f}, {upper:.1f}]')
    print()

Outlier Detection (using 3*IQR method):
temperature_celsius:
  Outliers: 112 (0.11%)
  Range: [-12.9, 59.2]

humidity:
  Outliers: 0 (0.00%)
  Range: [-60.0, 192.0]

wind_kph:
  Outliers: 95 (0.09%)
  Range: [-28.0, 52.5]

pressure_mb:
  Outliers: 163 (0.16%)
  Range: [986.0, 1042.0]



**Note**: Weather data naturally has extreme values (e.g., -40°C in polar regions, 50°C in deserts). These aren't errors, just natural variation. We'll keep them unless they're physically impossible.

In [8]:
# check for physically impossible values
print('Checking for impossible values:')
print(f'Humidity > 100%: {(df["humidity"] > 100).sum()}')
print(f'Humidity < 0%: {(df["humidity"] < 0).sum()}')
print(f'Temperature < -90°C: {(df["temperature_celsius"] < -90).sum()}')
print(f'Temperature > 60°C: {(df["temperature_celsius"] > 60).sum()}')
print(f'Pressure < 870mb: {(df["pressure_mb"] < 870).sum()}')  # lowest ever recorded ~870mb
print(f'Pressure > 1085mb: {(df["pressure_mb"] > 1085).sum()}')  # highest ~1085mb

Checking for impossible values:
Humidity > 100%: 0
Humidity < 0%: 0
Temperature < -90°C: 0
Temperature > 60°C: 0
Pressure < 870mb: 0
Pressure > 1085mb: 2


## Create Useful Derived Features

In [9]:
# temperature-humidity index (feels-like is already provided, but let's add heat index category)
df['temp_feels_diff'] = df['feels_like_celsius'] - df['temperature_celsius']

# wind chill effect (negative means wind makes it feel colder)
df['wind_effect'] = df['temp_feels_diff']

# categorize weather conditions
def categorize_condition(condition):
    condition = str(condition).lower()
    if any(word in condition for word in ['sunny', 'clear']):
        return 'Clear'
    elif any(word in condition for word in ['cloud', 'overcast']):
        return 'Cloudy'
    elif any(word in condition for word in ['rain', 'drizzle', 'shower']):
        return 'Rainy'
    elif any(word in condition for word in ['snow', 'sleet', 'ice']):
        return 'Snowy'
    elif any(word in condition for word in ['fog', 'mist']):
        return 'Foggy'
    elif any(word in condition for word in ['thunder', 'storm']):
        return 'Stormy'
    else:
        return 'Other'

df['weather_category'] = df['condition_text'].apply(categorize_condition)

print('Weather condition categories:')
print(df['weather_category'].value_counts())

Weather condition categories:
weather_category
Cloudy    43984
Clear     36544
Rainy     18355
Foggy      4325
Snowy       403
Stormy      252
Other         5
Name: count, dtype: int64


In [10]:
# air quality index category (using US EPA index)
def aqi_category(aqi):
    if aqi == 1:
        return 'Good'
    elif aqi == 2:
        return 'Moderate'
    elif aqi == 3:
        return 'Unhealthy for Sensitive'
    elif aqi == 4:
        return 'Unhealthy'
    elif aqi == 5:
        return 'Very Unhealthy'
    else:
        return 'Hazardous'

df['aqi_category'] = df['air_quality_us-epa-index'].apply(aqi_category)

print('\nAir quality distribution:')
print(df['aqi_category'].value_counts())


Air quality distribution:
aqi_category
Good                       53786
Moderate                   33174
Unhealthy for Sensitive     8905
Unhealthy                   6535
Very Unhealthy              1046
Hazardous                    422
Name: count, dtype: int64


## Save Cleaned Dataset

In [11]:
# save the cleaned data
df.to_csv('../data/weather_cleaned.csv', index=False)
print(f'Saved cleaned dataset: {len(df)} rows, {len(df.columns)} columns')
print(f'\nNew features added:')
new_features = ['year', 'month', 'day', 'hour', 'day_of_week', 'day_name', 
                'hemisphere', 'climate_zone', 'temp_feels_diff', 'wind_effect',
                'weather_category', 'aqi_category']
for feat in new_features:
    print(f'  - {feat}')

Saved cleaned dataset: 103868 rows, 53 columns

New features added:
  - year
  - month
  - day
  - hour
  - day_of_week
  - day_name
  - hemisphere
  - climate_zone
  - temp_feels_diff
  - wind_effect
  - weather_category
  - aqi_category


In [12]:
# show final dataset structure
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103868 entries, 0 to 103867
Data columns (total 53 columns):
 #   Column                        Non-Null Count   Dtype         
---  ------                        --------------   -----         
 0   country                       103868 non-null  object        
 1   location_name                 103868 non-null  object        
 2   latitude                      103868 non-null  float64       
 3   longitude                     103868 non-null  float64       
 4   timezone                      103868 non-null  object        
 5   last_updated_epoch            103868 non-null  int64         
 6   last_updated                  103868 non-null  datetime64[ns]
 7   temperature_celsius           103868 non-null  float64       
 8   temperature_fahrenheit        103868 non-null  float64       
 9   condition_text                103868 non-null  object        
 10  wind_mph                      103868 non-null  float64       
 11  wind_kph     

## Summary

**What we did:**
- ✅ Converted timestamps to datetime format
- ✅ Created time-based features (year, month, day, hour)
- ✅ Added geographic features (hemisphere, climate zones)
- ✅ Checked for outliers and impossible values
- ✅ Created derived features (weather categories, AQI labels)
- ✅ Saved cleaned dataset for analysis

**Next steps:** Exploratory Data Analysis to understand patterns and relationships in the data.